<a href="https://colab.research.google.com/github/jillybean8848/Amazon_Vine_Analysis/blob/main/Copy_of_Copy_of_Amazon_Reviews_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 261 kB in 3s (101 kB/s)
Reading package lis

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-12-10 19:21:01--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  5.32MB/s    in 0.2s    

2022-12-10 19:21:01 (5.32 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Beauty_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Beauty_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+---------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|          review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+---------------------+-----------+
|         US|    1797882|R3I2DHQBR577SS|B001ANOOOE|       2102612|The Naked Bee Vit...|          Beauty|          5|            0|          0|   N|                Y|          Five Stars| Love this, excell...| 2015-08-31|
|         US|   18381298|R1QNE9NQFJC2Y4|B0016J22EQ|     106393691|Alba Botanica Sun...|          Beauty|          5|

### Create DataFrames to match tables

In [ ]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
beauty_reviews_df = df.select(["review_id","customer_id","product_id","product_parent","product_title","review_date","star_rating", "helpful_votes","total_votes","vine","verified_purchase"])
beauty_reviews_df.show(10)

+--------------+-----------+----------+--------------+--------------------+-----------+-----------+-------------+-----------+----+-----------------+
|     review_id|customer_id|product_id|product_parent|       product_title|review_date|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+----------+--------------+--------------------+-----------+-----------+-------------+-----------+----+-----------------+
|R3I2DHQBR577SS|    1797882|B001ANOOOE|       2102612|The Naked Bee Vit...| 2015-08-31|          5|            0|          0|   N|                Y|
|R1QNE9NQFJC2Y4|   18381298|B0016J22EQ|     106393691|Alba Botanica Sun...| 2015-08-31|          5|            0|          0|   N|                Y|
|R3LIDG2Q4LJBAO|   19242472|B00HU6UQAG|     375449471|Elysee Infusion S...| 2015-08-31|          5|            0|          0|   N|                Y|
|R3KSZHPAEVPEAL|   19551372|B002HWS7RM|     255651889|Diane D722 Color,...| 2015-08-31|          5|       

In [ ]:
# Create the customers_table DataFrame
# customers_df = df.groupby("").agg({""}).withColumnRenamed("", "customer_count")
customers_df = beauty_reviews_df.groupby("customer_id").agg({"customer_id": 'count'}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show(10)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|    9015608|             3|
|    9767880|             2|
|   15958313|             1|
|   12879980|            20|
|   43920023|            31|
|     106535|             1|
|   20328662|             1|
|   12819130|             4|
|   50145309|             1|
|   15159322|             2|
|   16747249|             2|
|   37678162|             1|
|     227763|             1|
|   15191144|             2|
|   31946913|             1|
|   44295916|             1|
|   19021463|             1|
|     806337|             2|
|   16306618|            18|
|     393186|             1|
+-----------+--------------+
only showing top 20 rows



In [ ]:
products_df = beauty_reviews_df.select(["product_title", "product_id"])

In [ ]:
products_df.count()

5115666

In [ ]:
# Create the products_table DataFrame and drop duplicates. 
# products_df = df.select([]).drop_duplicates()
products_df = products_df.drop_duplicates(["product_id"])
products_df.show(20)

+--------------------+----------+
|       product_title|product_id|
+--------------------+----------+
|Henna Art, Tools ...|1935682016|
|Overnight Success...|9788077757|
|Dolce & Gabbana C...|9790770367|
|Calvin Klein Euph...|9790773587|
|Herve Leger Perfu...|9790796927|
|Trouble By Bouche...|9790800886|
|Women Perfume Oil...|9865962039|
|Conair 077B 1875-...|B000050B6Q|
|Pond's Overnight ...|B000052YQV|
|Neutrogena Rainba...|B000052ZBD|
|Wet 'n' Wild Crem...|B000052ZEO|
|Neutrogena Clean ...|B0000532VT|
|Sunbeam SBCL870 1...|B00005CDRS|
|Cover Girl Outlas...|B00005NAPK|
|Oral-B Sensitive ...|B00006ANDK|
|AXE Deodorant Bod...|B000089SAM|
|Philips HQ481 Tra...|B00008BFTP|
|Revlon ColorStay ...|B00008KA86|
|Aveeno Moisture L...|B0000A4EW2|
|Therapist Select ...|B0000AKT6Q|
+--------------------+----------+
only showing top 20 rows



In [ ]:
products_df.count()

588817

In [ ]:
beauty_reviews_df.count()

5115666

In [ ]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
# review_id_df = df.select([, to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df = beauty_reviews_df.select(["review_id","customer_id", "product_id","product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show(10)


+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3I2DHQBR577SS|    1797882|B001ANOOOE|       2102612| 2015-08-31|
|R1QNE9NQFJC2Y4|   18381298|B0016J22EQ|     106393691| 2015-08-31|
|R3LIDG2Q4LJBAO|   19242472|B00HU6UQAG|     375449471| 2015-08-31|
|R3KSZHPAEVPEAL|   19551372|B002HWS7RM|     255651889| 2015-08-31|
| RAI2OIG50KZ43|   14802407|B00SM99KWU|     116158747| 2015-08-31|
|R1R30FA4RB5P54|    2909389|B000NYL1Z6|     166146615| 2015-08-31|
|R30IJKCGJBGPJH|   19397215|B001SYWTFG|     111742328| 2015-08-31|
|R18GLJJPVQ1OVH|    3195210|B005F2EVMQ|     255803087| 2015-08-31|
| R8TVYIJXLYJT0|   52216383|B00M1SUW7K|     246816549| 2015-08-31|
|R1CJGF6M3PVHEZ|   10278216|B001KYQA1S|       9612905| 2015-08-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 10 rows



In [ ]:
# Create the vine_table. DataFrame
# vine_df = df.select([])
vine_df = beauty_reviews_df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.tail(10)

[Row(review_id='R1W18J8JHK3QT4', star_rating='5', helpful_votes=12, total_votes=16, vine='N', verified_purchase='N'),
 Row(review_id='RQLSGNI4JL4O7', star_rating='5', helpful_votes=36, total_votes=37, vine='N', verified_purchase='N'),
 Row(review_id='R3SHWVR8GO9WK4', star_rating='5', helpful_votes=26, total_votes=26, vine='N', verified_purchase='N'),
 Row(review_id='R2KNC11QJB7V5U', star_rating='5', helpful_votes=30, total_votes=34, vine='N', verified_purchase='N'),
 Row(review_id='R1OXK8PJ0TJ6UA', star_rating='4', helpful_votes=4, total_votes=4, vine='N', verified_purchase='N'),
 Row(review_id='RZ7RZ02MTP4SL', star_rating='5', helpful_votes=10, total_votes=10, vine='N', verified_purchase='N'),
 Row(review_id='R2IRC0IZ8YCE5T', star_rating='3', helpful_votes=23, total_votes=23, vine='N', verified_purchase='N'),
 Row(review_id='R1U4ZSXOD228CZ', star_rating='5', helpful_votes=89, total_votes=97, vine='N', verified_purchase='N'),
 Row(review_id='R3SFJLZE09URWM', star_rating='5', helpful_vo

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [ ]:
# Configure settings for RDS

from getpass import getpass

password = getpass("Enter PgAdmin password")

Enter PgAdmin password··········


In [ ]:
# Configure settings for RDS

mode = "append"
jdbc_url="jdbc:postgresql://amazon-vine-jillybean8848-cwru.cram7x4auom8.us-east-1.rds.amazonaws.com:5432/Amazon_Vine"
config = {"user":"root", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)